# Install findspark - Python Library


In [7]:
!pip install -q findspark

In [11]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/drive/My Drive/Colab Notebooks/IDS-561/HMW02/spark-3.0.1-bin-hadoop2.7"

# Step 3. Set environment variables
Set Java and Spark home based on the location where they are stored


In [12]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

# Reading both the files into data frame

In [13]:
df = spark.read.csv('Amazon_Responded_Oct05.csv', inferSchema=True, header=True)
df.show(5)

+--------------------+--------------------+----------------+-----------+-------------------+---------------------+--------------+-----------------+--------------+--------------------+-------------+-------------+--------------------+------------------+--------------------+--------------+--------------------+--------------+---------+-----------------------+-------------------------+-----------------------+-------------+---------+--------------------+
|              id_str|    tweet_created_at|user_screen_name|user_id_str|user_statuses_count|user_favourites_count|user_protected|user_listed_count|user_following|    user_description|user_location|user_verified|user_followers_count|user_friends_count|     user_created_at|tweet_language|               text_|favorite_count|favorited|in_reply_to_screen_name|in_reply_to_status_id_str|in_reply_to_user_id_str|retweet_count|retweeted|                text|
+--------------------+--------------------+----------------+-----------+-------------------+--

In [14]:
findDF = spark.read.csv('find_text.csv', inferSchema=True, header =True)
findDF.show(5)

+--------------------+----+
|              id_str|text|
+--------------------+----+
|'793270689780203520'|null|
|'793281386912354304'|null|
|'793299404975247360'|null|
|'793301295255945216'|null|
|'793315815411978240'|null|
+--------------------+----+
only showing top 5 rows



# Converting the data frame into RDD


In [15]:
rdd1 = df.rdd
rdd1.collect()

[Row(id_str="'793270689780203520'", tweet_created_at='Tue Nov 01 01:57:25 +0000 2016', user_screen_name='SeanEPanjab', user_id_str='143515471', user_statuses_count='51287', user_favourites_count='4079', user_protected='False', user_listed_count='74', user_following='False', user_description="Content marketer; Polyglot; Beard aficionado; Sikh. Persian, Catalan, French, Spanish. You'll find lol in the interstitial lulls of my tweets.", user_location='غریب الوطن', user_verified='False', user_followers_count='1503', user_friends_count='850', user_created_at='Thu May 13 17:43:52 +0000 2010', tweet_language='en', text_='@AmazonHelp Can you please DM me? A product I ordered last year never arrived.', favorite_count='0', favorited='False', in_reply_to_screen_name='AmazonHelp', in_reply_to_status_id_str=None, in_reply_to_user_id_str='85741735', retweet_count='0', retweeted='False', text=None),
 Row(id_str="'793281386912354304'", tweet_created_at='Tue Nov 01 02:39:55 +0000 2016', user_screen_nam

# Task 1

In [16]:
rdd2 = rdd1.map(lambda x: [x[i] for i in [0,1,11,16,17,22]])
rdd2.collect()

[["'793270689780203520'",
  'Tue Nov 01 01:57:25 +0000 2016',
  'False',
  '@AmazonHelp Can you please DM me? A product I ordered last year never arrived.',
  '0',
  '0'],
 ["'793281386912354304'",
  'Tue Nov 01 02:39:55 +0000 2016',
  'True',
  "@SeanEPanjab I'm sorry, we're unable to DM you. Was this order purchased on https://t.co/nUUp5MLhYl, or one of our other sites? ^CL",
  '0',
  '0'],
 ["'793501578766319616'",
  'Tue Nov 01 17:14:53 +0000 2016',
  'False',
  '@AmazonHelp It was purchased on https://t.co/gPWnuSkbwC.',
  '0',
  '0'],
 ["'793501657346682880'",
  'Tue Nov 01 17:15:12 +0000 2016',
  'False',
  '@AmazonHelp I am following you now, if it helps to DM.',
  '0',
  '0'],
 ["'793502854459879424'",
  'Tue Nov 01 17:19:57 +0000 2016',
  'True',
  '@SeanEPanjab Please give us a call/chat so we can look into this order for you: https://t.co/hApLpMlfHN. ^HB',
  '0',
  '0'],
 ["'793504235400884224'",
  'Tue Nov 01 17:25:26 +0000 2016',
  'True',
  '@SeanEPanjab Without providing

Filtering for users verified as True

In [17]:
rdd3 = rdd2.filter(lambda x: x[2] == 'True')
rdd3.collect()

[["'793281386912354304'",
  'Tue Nov 01 02:39:55 +0000 2016',
  'True',
  "@SeanEPanjab I'm sorry, we're unable to DM you. Was this order purchased on https://t.co/nUUp5MLhYl, or one of our other sites? ^CL",
  '0',
  '0'],
 ["'793502854459879424'",
  'Tue Nov 01 17:19:57 +0000 2016',
  'True',
  '@SeanEPanjab Please give us a call/chat so we can look into this order for you: https://t.co/hApLpMlfHN. ^HB',
  '0',
  '0'],
 ["'793504235400884224'",
  'Tue Nov 01 17:25:26 +0000 2016',
  'True',
  '@SeanEPanjab Without providing any sensitive information, do the order details indicate that the item was ever shipped? ^SW',
  '0',
  '0'],
 ["'793513446633533440'",
  'Tue Nov 01 18:02:03 +0000 2016',
  'True',
  "@SeanEPanjab I'm not able to access account info here. Please reach out by Phone/Chat so we can look at this: https://t.co/EKXRLsnxJu ^GL",
  '0',
  '0'],
 ["'793301295255945216'",
  'Tue Nov 01 03:59:02 +0000 2016',
  'True',
  '@aakashwangnoo Hi there, kindly write back to us with 

Counting the number of verified accounts

In [18]:
rdd3.count()

171797

Changing the Date Column Format

In [19]:
rdd4 = rdd3.map(lambda x: (x[0], x[1].split(' ')[1:3], x[2], x[3], x[4], x[5])).map(lambda x: (x[0], x[1][0]+x[1][1], x[2], x[3], x[4], x[5]))
rdd4.collect()

[("'793281386912354304'",
  'Nov01',
  'True',
  "@SeanEPanjab I'm sorry, we're unable to DM you. Was this order purchased on https://t.co/nUUp5MLhYl, or one of our other sites? ^CL",
  '0',
  '0'),
 ("'793502854459879424'",
  'Nov01',
  'True',
  '@SeanEPanjab Please give us a call/chat so we can look into this order for you: https://t.co/hApLpMlfHN. ^HB',
  '0',
  '0'),
 ("'793504235400884224'",
  'Nov01',
  'True',
  '@SeanEPanjab Without providing any sensitive information, do the order details indicate that the item was ever shipped? ^SW',
  '0',
  '0'),
 ("'793513446633533440'",
  'Nov01',
  'True',
  "@SeanEPanjab I'm not able to access account info here. Please reach out by Phone/Chat so we can look at this: https://t.co/EKXRLsnxJu ^GL",
  '0',
  '0'),
 ("'793301295255945216'",
  'Nov01',
  'True',
  '@aakashwangnoo Hi there, kindly write back to us with the details requested so we can proceed further with this.  ^KJ',
  '0',
  '0'),
 ("'793423313674571776'",
  'Nov01',
  'True

Sorting according the highest tweets per day

In [20]:
rdd5 = rdd4.groupBy(lambda x: x[1]).mapValues(lambda vals: len(set(vals)))
rdd6 = rdd5.sortBy(lambda x: x[1], ascending=False)
rdd6.collect()

[('Jan03', 1536),
 ('Jan10', 1508),
 ('Jan11', 1496),
 ('Jan12', 1410),
 ('Jan06', 1364),
 ('Jan07', 1360),
 ('Jan20', 1336),
 ('Mar02', 1296),
 ('Jan13', 1295),
 ('Jan21', 1290),
 ('Jan14', 1290),
 ('Jan18', 1286),
 ('Dec15', 1276),
 ('Jan24', 1259),
 ('Nov18', 1246),
 ('Dec03', 1197),
 ('Jan02', 1195),
 ('Jun27', 1191),
 ('Jul04', 1190),
 ('Jan19', 1175),
 ('Jan25', 1162),
 ('Jan23', 1149),
 ('Jan08', 1143),
 ('Jan17', 1124),
 ('May11', 1120),
 ('Jul03', 1112),
 ('Mar30', 1109),
 ('Apr05', 1088),
 ('Jan26', 1082),
 ('Jan27', 1080),
 ('Jun20', 1074),
 ('Jan04', 1071),
 ('Feb22', 1058),
 ('Nov05', 1058),
 ('Feb09', 1054),
 ('Jun23', 1045),
 ('Jun30', 1040),
 ('Jan22', 1024),
 ('May03', 1020),
 ('Mar08', 1019),
 ('Jul29', 1019),
 ('Mar29', 1018),
 ('Apr18', 1016),
 ('Jan16', 1007),
 ('Jan15', 1006),
 ('Jan09', 1003),
 ('Jun07', 996),
 ('Feb02', 987),
 ('Jun02', 982),
 ('Dec10', 979),
 ('Dec13', 979),
 ('Dec26', 978),
 ('Jan30', 968),
 ('Feb28', 968),
 ('Feb03', 960),
 ('Jan28', 955),
 (

In [21]:
rdd7 = rdd4.filter(lambda x: x[1] == 'Jan03')
rdd7.collect()

[("'816139627270721537'",
  'Jan03',
  'True',
  "@mcp111 Hey! We'll have this looked into internally. Thanks for your patience. ^HK",
  '0',
  '0'),
 ("'816357712732819467'",
  'Jan03',
  'True',
  "@mcp111 Hi there, you can change the country from the settings. Please reach us via the link provided earlier and we'll help you out, ^VM",
  '0',
  '0'),
 ("'816167583263162369'",
  'Jan03',
  'True',
  "@stevebrowne Hi Steve, sorry to hear this. As we can't view accounts here, please contact here for more options: https://t.co/JzP7hlA23B ^CJ",
  '0',
  '0'),
 ("'816326113660174336'",
  'Jan03',
  'True',
  "@boozysmurf We'd like the chance to look into this w/ you &amp; investigate further. Pls reach out by phone/chat: https://t.co/3pBuw1SV0L ^KD",
  '0',
  '0'),
 ("'816359349358575616'",
  'Jan03',
  'True',
  '@MeghanSuslak If you have the tracking ID from the shipping label, please contact us by phone or chat here: https://t.co/CYqkdUakWJ ^GL',
  '0',
  '0'),
 ("'816361236946223104'",

Filtering the RDD and summing up favourite_count and retweet_count

In [22]:
rdd8 = rdd7.filter(lambda x: x[4] is not None)
rdd9 = rdd8.filter(lambda x: x[5] is not None)
rdd10 = rdd9.filter(lambda x: x[4] != ' you may request a return here: https://t.co/S4eX6UOjIb ^MM"')
rdd11 = rdd10.map(lambda x: (x[0], x[1], x[2], x[3], int(x[4]) + int(x[5])))
rdd12 = rdd11.sortBy(lambda x: x[4], ascending=False)
rdd12.collect() 

[("'816329761530093568'",
  'Jan03',
  'True',
  '@amazon worst shopping  experience,  no service, no substantial reply to complaints, no delivery for 1 week post guarantee date.',
  5),
 ("'816083406962434048'",
  'Jan03',
  'True',
  '@ItsJosshA We always aim to deliver by the date given in your confirmation email. Have we missed that date? Any update in tracking?  ^NF',
  3),
 ("'816086117938319360'",
  'Jan03',
  'True',
  "@ItsJosshA Oh no! I'm sorry! Please reach out to us so that we can look into options: https://t.co/hApLpMlfHN. ^JO",
  2),
 ("'816095108013654017'",
  'Jan03',
  'True',
  '@KStefl Sounds like you know what to add to your Halloween playlist for this year! ^BV',
  2),
 ("'816109446069911554'",
  'Jan03',
  'True',
  '@Schoey1992 Happy Birthday, Matt! #FriendsForever #FriendshipGoals ^JO',
  2),
 ("'816157517428523008'",
  'Jan03',
  'True',
  '@ratbones666 You so fancy!!! You already knooow!!! ^EP',
  2),
 ("'816217909819297792'",
  'Jan03',
  'True',
  "@ThorpPe

Top 100 tweets for Jan03 sorted based on sum of retweet and favorite count

In [23]:
rdd13=rdd12.take(100)
print(rdd13[0][3])
len(rdd13)

@amazon worst shopping  experience,  no service, no substantial reply to complaints, no delivery for 1 week post guarantee date.


100

Only tweets copied into a new list inorder to perform word count 

In [24]:
rdd14=[row[3] for row in rdd13]

for i in rdd14:
    print(i)

@amazon worst shopping  experience,  no service, no substantial reply to complaints, no delivery for 1 week post guarantee date.
@ItsJosshA We always aim to deliver by the date given in your confirmation email. Have we missed that date? Any update in tracking?  ^NF
@ItsJosshA Oh no! I'm sorry! Please reach out to us so that we can look into options: https://t.co/hApLpMlfHN. ^JO
@KStefl Sounds like you know what to add to your Halloween playlist for this year! ^BV
@Schoey1992 Happy Birthday, Matt! #FriendsForever #FriendshipGoals ^JO
@ratbones666 You so fancy!!! You already knooow!!! ^EP
@ThorpPerrow Awww! Happy Birthday, you don't look a day over 20 ;-) .Pass on your details here: https://t.co/g52MRb0AOt for lil surprise ^RS
@thedexterouz Hi! We'd like to help. When you have a moment, please connect with us here:  https://t.co/vlvfJr4nN9 ^YP
@matt_linsley Please keep us posted on this and let us know if it doesn't arrive tomorrow. ^BV
@VlSlT Sorry to hear that, contact us here: https:/

In [26]:
#Data Cleaning: Removing Punctuation
import string
rdd14 = [''.join(c for c in s if c not in string.punctuation) for s in rdd14]

for i in rdd14:
    print(i)

amazon worst shopping  experience  no service no substantial reply to complaints no delivery for 1 week post guarantee date
ItsJosshA We always aim to deliver by the date given in your confirmation email Have we missed that date Any update in tracking  NF
ItsJosshA Oh no Im sorry Please reach out to us so that we can look into options httpstcohApLpMlfHN JO
KStefl Sounds like you know what to add to your Halloween playlist for this year BV
Schoey1992 Happy Birthday Matt FriendsForever FriendshipGoals JO
ratbones666 You so fancy You already knooow EP
ThorpPerrow Awww Happy Birthday you dont look a day over 20  Pass on your details here httpstcog52MRb0AOt for lil surprise RS
thedexterouz Hi Wed like to help When you have a moment please connect with us here  httpstcovlvfJr4nN9 YP
mattlinsley Please keep us posted on this and let us know if it doesnt arrive tomorrow BV
VlSlT Sorry to hear that contact us here httpstcohApLpMlfHN and well look into it for you AS
PPramod2041984 Hi apologies f

In [27]:
#Data cleaning: Converting to lower case
rdd15= [x.lower() for x in rdd14]

for i in rdd15:
    print(i)

amazon worst shopping  experience  no service no substantial reply to complaints no delivery for 1 week post guarantee date
itsjossha we always aim to deliver by the date given in your confirmation email have we missed that date any update in tracking  nf
itsjossha oh no im sorry please reach out to us so that we can look into options httpstcohaplpmlfhn jo
kstefl sounds like you know what to add to your halloween playlist for this year bv
schoey1992 happy birthday matt friendsforever friendshipgoals jo
ratbones666 you so fancy you already knooow ep
thorpperrow awww happy birthday you dont look a day over 20  pass on your details here httpstcog52mrb0aot for lil surprise rs
thedexterouz hi wed like to help when you have a moment please connect with us here  httpstcovlvfjr4nn9 yp
mattlinsley please keep us posted on this and let us know if it doesnt arrive tomorrow bv
vlslt sorry to hear that contact us here httpstcohaplpmlfhn and well look into it for you as
ppramod2041984 hi apologies f

In [28]:
#Data Cleaning: Removing Numbers
import re 
  
def remove(list): 
    pattern = '[0-9]'
    list = [re.sub(pattern, '', i) for i in list] 
    return list
rdd16=remove(rdd15)
for i in rdd16:
    print(i)

amazon worst shopping  experience  no service no substantial reply to complaints no delivery for  week post guarantee date
itsjossha we always aim to deliver by the date given in your confirmation email have we missed that date any update in tracking  nf
itsjossha oh no im sorry please reach out to us so that we can look into options httpstcohaplpmlfhn jo
kstefl sounds like you know what to add to your halloween playlist for this year bv
schoey happy birthday matt friendsforever friendshipgoals jo
ratbones you so fancy you already knooow ep
thorpperrow awww happy birthday you dont look a day over   pass on your details here httpstcogmrbaot for lil surprise rs
thedexterouz hi wed like to help when you have a moment please connect with us here  httpstcovlvfjrnn yp
mattlinsley please keep us posted on this and let us know if it doesnt arrive tomorrow bv
vlslt sorry to hear that contact us here httpstcohaplpmlfhn and well look into it for you as
ppramod hi apologies for the bad experience 

Converting the above list which contains the cleaned data into RDD

In [29]:
from pyspark.context import SparkContext
from pyspark import SparkConf
sc = SparkContext.getOrCreate()
rdd17=sc.parallelize(rdd16)
rdd17.take(5)

['amazon worst shopping  experience  no service no substantial reply to complaints no delivery for  week post guarantee date',
 'itsjossha we always aim to deliver by the date given in your confirmation email have we missed that date any update in tracking  nf',
 'itsjossha oh no im sorry please reach out to us so that we can look into options httpstcohaplpmlfhn jo',
 'kstefl sounds like you know what to add to your halloween playlist for this year bv',
 'schoey happy birthday matt friendsforever friendshipgoals jo']

In [30]:
rdd18 = rdd17.map(lambda x:x)
rdd18.collect()

['amazon worst shopping  experience  no service no substantial reply to complaints no delivery for  week post guarantee date',
 'itsjossha we always aim to deliver by the date given in your confirmation email have we missed that date any update in tracking  nf',
 'itsjossha oh no im sorry please reach out to us so that we can look into options httpstcohaplpmlfhn jo',
 'kstefl sounds like you know what to add to your halloween playlist for this year bv',
 'schoey happy birthday matt friendsforever friendshipgoals jo',
 'ratbones you so fancy you already knooow ep',
 'thorpperrow awww happy birthday you dont look a day over   pass on your details here httpstcogmrbaot for lil surprise rs',
 'thedexterouz hi wed like to help when you have a moment please connect with us here  httpstcovlvfjrnn yp',
 'mattlinsley please keep us posted on this and let us know if it doesnt arrive tomorrow bv',
 'vlslt sorry to hear that contact us here httpstcohaplpmlfhn and well look into it for you as',
 'pp

Splitting the RDD based on " " in order to get list of words

In [31]:
rdd19=rdd18.flatMap(lambda x: x.split(" "))
rdd19.collect()

['amazon',
 'worst',
 'shopping',
 '',
 'experience',
 '',
 'no',
 'service',
 'no',
 'substantial',
 'reply',
 'to',
 'complaints',
 'no',
 'delivery',
 'for',
 '',
 'week',
 'post',
 'guarantee',
 'date',
 'itsjossha',
 'we',
 'always',
 'aim',
 'to',
 'deliver',
 'by',
 'the',
 'date',
 'given',
 'in',
 'your',
 'confirmation',
 'email',
 'have',
 'we',
 'missed',
 'that',
 'date',
 'any',
 'update',
 'in',
 'tracking',
 '',
 'nf',
 'itsjossha',
 'oh',
 'no',
 'im',
 'sorry',
 'please',
 'reach',
 'out',
 'to',
 'us',
 'so',
 'that',
 'we',
 'can',
 'look',
 'into',
 'options',
 'httpstcohaplpmlfhn',
 'jo',
 'kstefl',
 'sounds',
 'like',
 'you',
 'know',
 'what',
 'to',
 'add',
 'to',
 'your',
 'halloween',
 'playlist',
 'for',
 'this',
 'year',
 'bv',
 'schoey',
 'happy',
 'birthday',
 'matt',
 'friendsforever',
 'friendshipgoals',
 'jo',
 'ratbones',
 'you',
 'so',
 'fancy',
 'you',
 'already',
 'knooow',
 'ep',
 'thorpperrow',
 'awww',
 'happy',
 'birthday',
 'you',
 'dont',
 'lo

Applying map, reduceByKey and sortBy functions to get the final word count sorted in ascending order.

In [32]:
# count the occurrence of each word
wordCounts = rdd19.map(lambda word: (word, 1)).reduceByKey(lambda a,b:a+b).sortBy(lambda x:x,ascending=True)
wordCounts.take(5)

for pair in wordCounts.take(100):
    print(pair)

('', 29)
('a', 24)
('ab', 1)
('abhinavsaroj', 4)
('able', 3)
('about', 4)
('aboutasix', 1)
('above', 2)
('access', 3)
('account', 2)
('accttypically', 1)
('add', 1)
('address', 1)
('adibaddy', 1)
('advise', 1)
('ae', 3)
('af', 2)
('aim', 2)
('ak', 1)
('alerts', 2)
('alexa', 1)
('alexisgraham', 1)
('allow', 1)
('already', 1)
('always', 3)
('alyssagoldman', 3)
('am', 1)
('amazon', 7)
('amazonhelp', 1)
('amazonin', 2)
('ambermullennn', 1)
('amp', 1)
('an', 7)
('and', 17)
('any', 8)
('anything', 1)
('apologies', 2)
('app', 6)
('application', 1)
('appreciation', 1)
('ar', 3)
('are', 2)
('arent', 1)
('ariannadelbene', 1)
('arrange', 1)
('arrive', 6)
('arrived', 3)
('arrives', 1)
('as', 4)
('ask', 1)
('asked', 1)
('assist', 2)
('assistance', 1)
('at', 6)
('athena', 1)
('athenarivera', 1)
('atoz', 1)
('attention', 2)
('availability', 1)
('available', 2)
('aw', 1)
('awww', 1)
('babajabalpuri', 4)
('back', 1)
('bad', 2)
('baileyoz', 1)
('bayou', 1)
('bbyuniversecb', 1)
('be', 8)
('becoming', 1)


In [33]:
wordCounts.coalesce(1).saveAsTextFile("HMW2_FinalOutput.csv")

# Task 2

Converting data frame of find_test in to rdd

In [34]:
rddF=findDF.rdd

Mapping rdd of Amazon for id_str and text column

In [35]:
rdd13 = rdd1.map(lambda x: [x[i] for i in [0,16]])
rdd13.collect()

[["'793270689780203520'",
  '@AmazonHelp Can you please DM me? A product I ordered last year never arrived.'],
 ["'793281386912354304'",
  "@SeanEPanjab I'm sorry, we're unable to DM you. Was this order purchased on https://t.co/nUUp5MLhYl, or one of our other sites? ^CL"],
 ["'793501578766319616'",
  '@AmazonHelp It was purchased on https://t.co/gPWnuSkbwC.'],
 ["'793501657346682880'",
  '@AmazonHelp I am following you now, if it helps to DM.'],
 ["'793502854459879424'",
  '@SeanEPanjab Please give us a call/chat so we can look into this order for you: https://t.co/hApLpMlfHN. ^HB'],
 ["'793504235400884224'",
  '@SeanEPanjab Without providing any sensitive information, do the order details indicate that the item was ever shipped? ^SW'],
 ["'793511847899070465'",
  '@AmazonHelp It was shipped and possibly delivered to my old place of employment. But when I asked for it, no one had it. And so I wonder +'],
 ["'793511899279208449'", '@AmazonHelp if it was lost. I have the order #.'],
 ["

Joining the two RDD's on id_str

In [36]:
final=rdd13.join(rddF)
final.collect()

[("'793511847899070465'",
  ('@AmazonHelp It was shipped and possibly delivered to my old place of employment. But when I asked for it, no one had it. And so I wonder +',
   None)),
 ("'793407430344310785'",
  ('@AmazonHelp How many times do you want to write back to you guys??? Check the complaints through email regarding  Order 171-1338898-5999507.',
   None)),
 ("'793538125884645376'",
  ('@AmazonHelp @amazonIN @JeffBezos @consumeraction @ConsumerAffairs @ConsumersUnion and these are my responses. https://t.co/vTwhtX5oBt',
   None)),
 ("'793538326653394945'",
  ('@AmazonHelp @amazonIN @JeffBezos @consumeraction @ConsumerAffairs @ConsumersUnion Waiting for Amazon Team Response.',
   None)),
 ("'793551822476705793'",
  ("@aakashwangnoo Hey! I see that we've already responded to your query. That is the best possible resolution we can provide 1/2 ^MM",
   None)),
 ("'793551823147765765'",
  ('@aakashwangnoo at the moment. Appreciate your patience and understanding. 2/2 ^MM',
   None)),


Forming a new RDD

In [37]:
rdd_out = final.map(lambda x: (x[0], x[1][0]))
rdd_out.collect()

[("'793511847899070465'",
  '@AmazonHelp It was shipped and possibly delivered to my old place of employment. But when I asked for it, no one had it. And so I wonder +'),
 ("'793407430344310785'",
  '@AmazonHelp How many times do you want to write back to you guys??? Check the complaints through email regarding  Order 171-1338898-5999507.'),
 ("'793538125884645376'",
  '@AmazonHelp @amazonIN @JeffBezos @consumeraction @ConsumerAffairs @ConsumersUnion and these are my responses. https://t.co/vTwhtX5oBt'),
 ("'793538326653394945'",
  '@AmazonHelp @amazonIN @JeffBezos @consumeraction @ConsumerAffairs @ConsumersUnion Waiting for Amazon Team Response.'),
 ("'793551822476705793'",
  "@aakashwangnoo Hey! I see that we've already responded to your query. That is the best possible resolution we can provide 1/2 ^MM"),
 ("'793551823147765765'",
  '@aakashwangnoo at the moment. Appreciate your patience and understanding. 2/2 ^MM'),
 ("'793670911635312640'",
  "@AmazonHelp @amazonIN @JeffBezos @con

Converting RDD to CSV

In [39]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)
df = sqlContext.createDataFrame(rdd_out, ['id_str', 'text'])
# Write CSV (I have HDFS storage)
df.coalesce(1).write.format('com.databricks.spark.csv').options(header='true').save('final_test.csv')

In [40]:
df.show()

+--------------------+--------------------+
|              id_str|                text|
+--------------------+--------------------+
|'793511847899070465'|@AmazonHelp It wa...|
|'793407430344310785'|@AmazonHelp How m...|
|'793538125884645376'|@AmazonHelp @amaz...|
|'793538326653394945'|@AmazonHelp @amaz...|
|'793551822476705793'|@aakashwangnoo He...|
|'793551823147765765'|@aakashwangnoo at...|
|'793670911635312640'|@AmazonHelp @amaz...|
|'793322433625415680'|@Murtazansp Could...|
|'795207462160896001'|@Murtazansp Hey t...|
|'793397280254472193'|@anchitkhar HI! W...|
|'793524235306053632'|                null|
|'793826751499603968'|@imkapsicum Hey, ...|
|'793408494649081856'|@mybharatraj Hi, ...|
|'793412238073815040'|@AmazonHelp due t...|
|'793450914585739266'|@mybharatraj Hey!...|
|'793451025101389824'|@mybharatraj Kind...|
|'794600719697514496'|@mybharatraj Sorr...|
|'793383832720474113'|@praveen_pandey_ ...|
|'793847761393311745'|@AmazonHelp 3rd t...|
|'793926534742638592'|@praveen_p